## Spark graph

Dans les tutoriels précédents nous avons toujours travaillé avec des dataset ou table.

Il existe un écosystème spark pour traiter de gros graphes via l'api **graphx** historiquement que disponible en scala il existe **graphFrames** en python.

Essayons de jouer avec cette api.


### Création du context Spark

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
import os

conf = SparkConf()

#url par défaut d'une api kubernetes accédé depuis l'intérieur du cluster (ici le notebook tourne lui même dans kubernetes)
conf.setMaster("k8s://https://kubernetes.default.svc:443")

#image des executors spark: pour des raisons de simplicité on réutilise l'image du notebook
conf.set("spark.kubernetes.container.image", os.environ['IMAGE_NAME'])

# Nom du compte de service pour contacter l'api kubernetes : attention le package du datalab crée lui même cette variable d'enviromment.
# Dans un pod du cluster kubernetes il faut lire le fichier /var/run/secrets/kubernetes.io/serviceaccount/token
# Néanmoins ce paramètre est inutile car le contexte kubernetes local de ce notebook est préconfiguré
# conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) 

# Nom du namespace kubernetes
conf.set("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])

# Nombre d'executeur spark, il se lancera autant de pods kubernetes que le nombre indiqué.
conf.set("spark.executor.instances", "10")

# Mémoire alloué à la JVM
# Attention par défaut le pod kubernetes aura une limite supérieur qui dépend d'autres paramètres.
# On manipulera plus bas pour vérifier la limite de mémoire totale d'un executeur
conf.set("spark.executor.memory", "4g")

conf.set("spark.kubernetes.driver.pod.name", os.environ['KUBERNETES_POD_NAME'])

# Paramètres d'enregistrement des logs spark d'application
# Attention ce paramètres nécessitent la création d'un dossier spark-history. Spark ne le fait pas lui même pour des raisons obscurs
# import s3fs
# endpoint = "https://"+os.environ['AWS_S3_ENDPOINT']
# fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': endpoint})
# fs.touch('s3://tm8enk/spark-history/.keep')
# sparkconf.set("spark.eventLog.enabled","true")
# sparkconf.set("spark.eventLog.dir","s3a://tm8enk/spark-history")
#ici pour gérer le dateTimeFormatter dépendant de la verion de java...
conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
conf.set("spark.default.parallelism",10)
conf.set("spark.sql.shuffle.partitions",10)
conf.set("spark.jars.packages","graphframes:graphframes:0.8.1-spark3.0-s_2.12")


On note que :
* on a pris 10 executeurs et on surcharge spark pour, lors de shuffle ou repartition, que son niveau de parallelisme soit 10 plutot que 200
* on importe un jar au démarrage il va aller le chercher sur maven central, ce jar sera appellé par la librairie python graphFrames à travers Py4j.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("graph").config(conf = conf).getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4b2d4557-b063-420a-a465-474cfa3a9511;1.0
	confs: [default]
	found graphframes#graphframes;0.8.1-spark3.0-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.1-spark3.0-s_2.12/graphframes-0.8.1-spark3.0-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.1-spark3.0-s_2.12!graphframes.jar (70ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (19ms)
:: resolution report :: resolve 3496ms :: artifacts dl 93ms
	:: modules in use:
	graphframes#graphframes;0.8.1-spark3.0-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	-----------

In [3]:
%pip install graphframes

     |████████████████████████████████| 154 kB 7.0 MB/s            
Note: you may need to restart the kernel to use updated packages.


### Constitution d'un graphe

Un graphe peut être vu comme 2 datasets:
* Un **dataset de noeud** avec 2 colonnes : un identifiant et un attribut.
* Un **dataset d'arcs** reliant 2 noeuds : avec un identifiant de noeud source, un destination et un ou des attributs sur cet arc
 
A partir de ces 2 datasets que spark peut traiter comme des rdd à travers le cluster, on peut travailler sur un graphe, voici un exemple simple :

In [4]:
from graphframes import *

vertices = [(1,"A"), (2,"B"), (3, "C")]
edges = [(1,2,"love"), (2,1,"hate"), (2,3,"follow")]

v = spark.createDataFrame(vertices, ["id", "name"])
e = spark.createDataFrame(edges, ["src", "dst", "action"])

premierGraphe = GraphFrame(v, e)

In [5]:
premierGraphe.edges.show()

+---+---+------+
|src|dst|action|
+---+---+------+
|  1|  2|  love|
|  2|  1|  hate|
|  2|  3|follow|
+---+---+------+



### Faisons un graphe sur les données twitter concernant l'insee

Attention les données sont alimentées et à la réexecution du notebook les éléments suivants auront peut-être évolués, on peut filtrer sur la date des tweets si nécessaire avant le 16 avril pour retrouver le déroulé ci-dessous

In [6]:
#on importe le schema
import pickle
schema = pickle.load( open( "../7-spark-streaming/schema.p", "rb" ) )

In [8]:
#on lit les donnnées
df = spark.read.format("json")  \
    .schema(schema) \
    .load("s3a://projet-spark-lab/diffusion/tweets/input")

2022-03-30 07:22:20,319 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Essayons de créer le graph des tweets mentionnant l'insee avec comme noeud les user de twitter et comme arc, 2 utilisateurs sont reliés si l'émetteur du tweet a metionné un autre utilisateur dans le tweet

Par exemple si alice tweet "@bob tu vas vu le dernier tweet de l'Insee" le noeud alice sera relié -> à bob.

Nous avons de la chance les données de l'api twitter prémache le travail.

Regardons pour commencer dans le tweet de l'api twitter les données suivantes:
* l'identififiant du tweet
* l'identifiant de l'utilisateur qui tweete
* son nombre de followers pour savoir s'il est influent
* la liste des hashtags présents dans le tweet (déjà donné par twitter)
* une indicatrice pour savoir s'il y a des hashtag 
* le texte du tweet
* les user mentionnés dans le tweet sous forme de tableau (identifiant, name, screen_name)

In [10]:
from pyspark.sql.functions import col,explode,size,first
#.select("id","user.id","user.name","user.followers_count","entities.hashtags","text")
df.select(col("id").alias("id"),
         col("user.id").alias("user_id"),
         col("user.name").alias("name"),
         col("user.followers_count"),
         col("entities.hashtags.text").alias("hashtags"),
         size("entities.hashtags").alias("has_hashtag"),
         col("text"),
         col("entities.user_mentions")) \
 .filter(col("has_hashtag")>0) \
 .head(1) 

[Row(id=1380403542725365762, user_id=1229160600737132544, name='FREDERIC 🇫🇷🐷🍴', followers_count=1981, hashtags=['FakeNews'], has_hashtag=1, text='RT @ThierryMARIANI: Quand @MLP_officiel le dénonce, c’est une #FakeNews.\nQuand l’@InseeFr le confirme, cela devient hélas une évidence que…', user_mentions=[Row(id=87212906, id_str='87212906', indices=[3, 18], name='Thierry MARIANI', screen_name='ThierryMARIANI'), Row(id=217749896, id_str='217749896', indices=[26, 39], name='Marine Le Pen', screen_name='MLP_officiel'), Row(id=217473382, id_str='217473382', indices=[81, 89], name='Insee', screen_name='InseeFr')])]

Pour créer le dataset des noeuds:
* on récupére les id et noms des émetteurs de tweets

In [11]:
#vertices
tweeters=df.select(col("user.id").alias("id"), col("user.name").alias("name"))

* On récupère les id et noms des user mentionnés (même s'ils n'ont pas forcément twittés)

In [12]:
users_mentionned=df.select(explode(col("entities.user_mentions"))).select(col("col.id").alias("id"),col("col.screen_name").alias("name")).distinct()

On fait l'union des 2 dataframe et on va ne retenir qu'un nom (le nom dans la mention n'est pas toujours exact au nom du compte apparemment ca évitera les doublons)

In [13]:
vertices=tweeters.union(users_mentionned)

In [14]:
from pyspark.sql.functions import udf, collect_list
# on définit un udf qui prend le premier élément
udf_first = udf(lambda x: x[0])
# on groupe l'union des deux dataset par id d'utilisateur et on collect 
#sous la forme d'une liste leurs noms qui peuvent différer entre les mentions et le compte
# on applique l'udf pour ne retenir que le nom en tête

final_vertices=vertices.groupby("id").agg(udf_first(collect_list("name")).alias("name"))

In [15]:
final_vertices.show(10, truncate=False)

+-------+------------------------+
|id     |name                    |
+-------+------------------------+
|612473 |BBCNews                 |
|707913 |Fender                  |
|1012981|Jean-Yves Stervinou     |
|1162091|PierreCol               |
|1349111|Croquignol              |
|1536651|Rubin Sfadj             |
|1745171|Charles                 |
|1994321|FRANCE24                |
|2357391|Christophe Prieuur      |
|4478161|Adam Curtis stan account|
+-------+------------------------+
only showing top 10 rows



Faisons maintenant le dataset des arcs :
* on prend l'identifiant de l'émetteur
* on prend duplique/explose la ligne pour avoir une ligne par mention dans le tweet
* on récupère les hashtags
* on récupère l'identifiant du tweet

on group by user_id et mention.id (user mentionné) et on agggrège pour avoir :
* le nombre de tweets
* la liste des hashtags vus entre ces 2 users dans le sens user_id -> mention.id

In [16]:
from pyspark.sql.functions import count,collect_list,flatten
edges=df.select(col("user.id").alias("user_id"), \
               explode(col("entities.user_mentions")).alias("mention"),
               col("entities.hashtags.text").alias("hashtags"),
               "id") \
 .groupby(col("user_id").alias("src"), \
          col("mention.id").alias("dst")) \
 .agg(count("id").alias("nb"),
      flatten(collect_list("hashtags")).alias("hashtags"),
      collect_list("id").alias("id"))

In [17]:
edges.show()

+-------+-------------------+---+--------+--------------------+
|    src|                dst| nb|hashtags|                  id|
+-------+-------------------+---+--------+--------------------+
|1012981| 973130412628246529|  1|      []|[1385318639994290...|
|1349111|          217473382|  1|      []|[1426382322706919...|
|1536651|           17510568|  1|      []|[1382995514933850...|
|1745171|           16649457|  1|      []|[1436253710137667...|
|3639301|         4041503175|  1|      []|[1386964092682964...|
|4478161|          217473382|  1|      []|[1433405191211130...|
|4478161|         3077516146|  1|      []|[1394763627283066...|
|5523462|          268227446|  1|      []|[1394285638040567...|
|5891532|            5788732|  1|      []|[1394724638886875...|
|6274062|          200659061|  1|      []|[1425104376738336...|
|6465822|         2832878397|  1|      []|[1386338289511280...|
|6465822|1194947441734361089|  1|      []|[1386338289511280...|
|6504012|          393484793|  1|      [

### Créer le graphe

Ca y est on a nos 2 structures:
* **identifiant**, name ici seul **identifiant** est nécessaire name vient donner plus d'infos on pourrait ajouter d'autres colonnes
* **identifianttwittant(src), identifiantmentionné(dst)**, nombre de fois, liste de hashtags, liste des ids de tweets ici seul les 2 premiers identifiants sont nécessaires le reste donne du contexte qu'on pourrait utiliser sur les arcs.


In [18]:
from graphframes import *
g = GraphFrame(final_vertices, edges)

Maintenant qu'on a ce graphe on va le mettre en cache car nous allons le manipuler plusieurs fois

In [19]:
g.cache()

GraphFrame(v:[id: bigint, name: string], e:[src: bigint, dst: bigint ... 3 more fields])

In [25]:
print("le graphe a "+ str(g.vertices.count()) +" noeuds et "+ str(g.edges.count()) + " arcs")

le graphe a 46124 noeuds et 75911 arcs


### Recherche des utilisateurs populaires

On peut via l'api appliquer différents algorithmes, il est d'usage dans les graphes de traiter des notions suivantes :
* la notion de degré (nombre de connexion d'un noeud)
* la notion de triangle (triplet de noeud totalement connecté)
* la notion de chemin entre noeud

L'algorithme pageRank est un algorithme qui est connu pour etre utilisé dans les moteurs de recherche, il peut etre executé sur un graphe pour juger de la "popularité" d'un noeud.

On s'attend à ce que le comte de l'insee soit pas mal placé.


In [26]:
pagerank = g.pageRank(resetProbability=0.15,tol=0.01)

In [27]:
from pyspark.sql.functions import desc
pagerank.vertices.sort(desc("pagerank")).show(truncate=False)

+-------------------+--------------------------------------------------+------------------+
|id                 |name                                              |pagerank          |
+-------------------+--------------------------------------------------+------------------+
|217473382          |Insee                                             |2006.2580214097795|
|304576847          |maximetandonnet                                   |753.6780909654278 |
|1312779074088194049|𝕏𝟚𝔻𝔼𝕄🇹🇬🇬🇭                                |423.54102917543133|
|103918784          |davidlisnard                                      |372.26630218296174|
|546941531          |Guillaume Duval                                   |346.82728313325487|
|983334079981654016 |Docteur Peter EL BAZE                             |336.18135734295106|
|1214315619031478272|Mediavenir                                        |335.22836144910286|
|1499400284         |J_Bardella                                        |317.9490499232238

Regardons pourquoi David Lisnard le maire de Cannes arrive dans le top des influenceurs

In [28]:
pagerank.edges.where((col("src")=="103918784") ).show(truncate=False)

+---------+---------+---+--------+---------------------------------------------------------------+------+
|src      |dst      |nb |hashtags|id                                                             |weight|
+---------+---------+---+--------+---------------------------------------------------------------+------+
|103918784|103918784|3  |[]      |[1380509496284430336, 1380509496284430336, 1380509496284430336]|0.5   |
|103918784|114222231|1  |[]      |[1430429793301024771]                                          |0.5   |
+---------+---------+---+--------+---------------------------------------------------------------+------+



In [29]:
pagerank.edges.where((col("dst")=="103918784") ).count()

545

David lisnard n'a tweeté qu'une fois mais il s'est fait mentionné 545 fois autour de ce tweet.

https://twitter.com/davidlisnard/status/1380509496284430336

On peut regarder les utilisateurs ayant le plus été mentionnés

In [30]:
g.inDegrees.join(g.vertices,"id").orderBy(desc("inDegree")).show(10,False)

+-------------------+--------+--------------------------------------------------+
|id                 |inDegree|name                                              |
+-------------------+--------+--------------------------------------------------+
|217473382          |5159    |Insee                                             |
|304576847          |2660    |maximetandonnet                                   |
|1499400284         |1163    |J_Bardella                                        |
|1214315619031478272|1115    |Mediavenir                                        |
|1312779074088194049|983     |𝕏𝟚𝔻𝔼𝕄🇹🇬🇬🇭                                |
|983334079981654016 |883     |Docteur Peter EL BAZE                             |
|2305392175         |692     |superflameur                                      |
|945473418          |632     |Docteur Laurent Alexandre #JeSuisDoublementVacciné|
|204368725          |620     |Ian Brossat                                       |
|978184280          |605 

ou qui ont mentionné le plus

In [31]:
g.outDegrees.join(g.vertices,"id").orderBy(desc("outDegree")).show(10,False)

+-------------------+---------+--------------------------------------------------+
|id                 |outDegree|name                                              |
+-------------------+---------+--------------------------------------------------+
|1312779074088194049|150      |𝕏𝟚𝔻𝔼𝕄🇹🇬🇬🇭                                |
|891710670856753152 |125      |infos etc                                         |
|217473382          |109      |Insee                                             |
|1107619640409276417|92       |Trinpk🍓✊ 0.1% #ResistSR                          |
|1359816991033458689|91       |Cris Finland                                      |
|1209500610787196929|68       |Observatoire de l'immigration et de la démographie|
|702991804661112832 |56       |Ari Kouts                                         |
|1378687476697550852|53       |BouleBille                                        |
|1222491562824949763|53       |Jacques                                           |
|1419627065809

L'api propose d'autres algorithmes permettant:
* de clusteriser le graphe (connected ou strong connected components)
* de rechercher des patterns ou chemin dans le graphe ce qui pourrait etre intéressant pour voir par exemple une chaine de retweet.


Enfin on peut nous même créer nos propres algorithmes pour cela l'api propose une méthode aggregate messages permettant d'envoyer un message de noeud et noeud et définir l'aggregation à retenir

In [32]:
spark.stop()

2022-03-30 08:05:36,267 WARN k8s.ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed.
